# **Práctica 2. Procesamiento de datos mediante Apache Spark**

**Autores**:

- Carlos Vigil González                 100363974
- David Gil López                       100363815
- Daniel Alejandro Rodríguez López      100316890


# Introducción 


En este trabajo se van a analizar una serie de datos sobre los Taxis de la Ciudad de Nueva York utilizando Apache Spark y Python (PySpark), todo ello ejecutado en este Jupyter Notebook.

***
En este caso se ha utilizado la versión 3 de Apache Spark para el desarrollo del código y sus pruebas.
***

En los apartados siguientes se van a ir comentando diferentes aspectos de la práctica, empezando con una limpieza de los datos de entrada, continuando con los diferentes análisis de dichos datos, para acabar con unas conclusiones sobre los tiempos de ejecución de cada estudio y sus velocidades, comparándolas entre si para ver por qué unos se ejecutan más rápido que otros.

# Código para ejecutar en google collab

***
Aquí dejamos el código para ejecutar el notebook en google collab. Para ello, descomentar el bloque de abajo y quitar el "import os" repetido en el bloque de Imports.
***

In [ ]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
#!tar xf /content/spark-3.0.1-bin-hadoop2.7.tgz
#!pip install -q findspark
#
#import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

# Imports

In [ ]:
import sys, os
is_conda = os.path.exists(os.path.join(sys.prefix, 'conda-meta'))

if not is_conda:
    import findspark 
    findspark.init()

from IPython.core.display import display, HTML
from IPython.display import IFrame
from collections import defaultdict
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, unix_timestamp
from time import time
import csv
import matplotlib.pyplot as plt
import numpy as np

# Para una lectura más distendida de la memoria
MODO_JAJAS = True

# Lectura de datos

En este apartado se incluye una función para facilitar la creación de las posteriores gráficas y la creación del entorno de spark con la lectura del CSV inicial.

In [ ]:
def autolabel(rects):
    # https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points", fontsize=25,
                    ha='center', va='bottom')

In [ ]:
if MODO_JAJAS:
    display(IFrame("https://giphy.com/embed/I1U9DTjCqOF3i",width="240", height="135"))

In [ ]:
spark = SparkSession.builder.appName("taxis").master("local[*]").getOrCreate()

In [ ]:
t0 = time()
df = spark.read.csv('./tripdata_2017_01.csv', header=True, inferSchema=True)

In [ ]:
df.printSchema()
dfP=df.toPandas()

# Limpieza de datos

En este apartado se limpian los datos de entrada, para qeu a la hora de procesarlos no salgan valores fuera de lo esperado.

In [ ]:
if MODO_JAJAS:
    display(IFrame("https://giphy.com/embed/xsATxBQfeKHCg", width="240", height="180"))

In [ ]:
initial_data_count = df.count()
print("Cantidad de datos usados:", initial_data_count)
display(dfP)
display(dfP.describe().T)

### Elementos extraños en el dataset

Lista de comportamientos extraños en los datos, y por tanto, inválidos a la hora de utilizarlos, ya que deberían ser coherentes basándonos en la información de cada campo proporcionada por la [documentación](https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)

* Existen carreras en las que la distancia es 0
* Existen propinas negativas
* "extra" con valores diferentes a 0 (ya que puede no haber extras), 0.5 y 1
* Existen viajes con un precio final negativo
* "MTA_tax" debe valer siempre 0.50. Valores diferentes son erróneos, y por tanto puede que el resto de la información también
* "Improvement_surcharge" es un valor en desuso, por lo que debería valer en el menor caso 0, no -0.3
* Carreras cuya fecha de fin sea igual o anterior a la fecha de inicio
* Existen tarifas con valores negativos. No tiene sentido ya que la tarifa va en función del tiempo y la distancia recorridas

### Elementos extraños PERO posibles

* Número de pasajeros es 0. Dado que es un valor que introduce el propio conductor, valores como 0 son posibles si al conductor le da igual introducir bien el valor.
* Un viaje empieza y acaba en la misma zona. Puede deberse a zonas grandes o que son viajes de ida y vuelta.



### Limpieza realizada

A partir de los comportamientos observados se ha procedido a eliminar las carreras que cumplen las siguientes condiciones:

- Campo "tip_amount" con valores menor a 0
- Campo "total_amount" con valores menor o igual a 0
- Campo "trip_distance" con valores menor o igual a 0
- Campo "fare_amount" con valores menor o igual a 0
- Campo "extra" con valores diferentes de 0, 0.5 y 1
- Campo "MTA_tax" con valor distinto de 0.5
- Campo "Improvement_surcharge" con valor distinto de 0 o 0.3
- Campo "tpep_dropoff_datetime" es anterior o igual a "tpep_pickup_datetime"

In [ ]:
# Convertimos las fechas a timestamp, para que dejen de ser strings a secas
# y guardamos su diferencia para luego tener más fácil el filtrado y otros cálculos

df = df.withColumn(
    "tpep_pickup_timestamp", unix_timestamp(col("tpep_pickup_datetime").cast("timestamp"))
).withColumn(
    "tpep_dropoff_timestamp", unix_timestamp(col("tpep_dropoff_datetime").cast("timestamp"))
).withColumn(
    "time_diff", col("tpep_dropoff_timestamp") - col("tpep_pickup_timestamp")  # Segundos
)

df.createOrReplaceTempView('datosCarreras')

In [ ]:
datosLimpios = spark.sql("""
    SELECT * FROM datosCarreras WHERE
        tip_amount >= 0 AND
        total_amount > 0 AND
        trip_distance > 0 AND
        fare_amount > 0 AND
        (extra == 0 OR extra == 0.5 OR extra == 1) AND
        mta_tax == 0.5 AND
        improvement_surcharge >= 0 AND
        time_diff > 0
""")
datosLimpios_count = datosLimpios.count()
datosLimpios.createOrReplaceTempView('datosCarrerasLimpios')


In [ ]:
# display(datosLimpiosP)
# display(datosLimpiosP.describe().T)

## Extracción de información

Ahora que ya hemos limpiado los datos y tenemos entradas coherentes, se puede proceder a extraer información de los mismos. 

La información que se va a extraer es:

* Velocidad media de los taxis en función de la hora.
* Viajes en taxi más comunes
* Registros financieros (propinas, personas, etc.)
    * Timos a turistas
    * Propinas en función de la hora
    * Identificar pasajeros borrachos
* Zonas con poca cobertura



In [ ]:
# Bajamos el csv con la información de las zonas para luego poder "traducir"
if not os.path.exists("taxi+_zone_lookup.csv"):
    !wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

In [ ]:
# Se añade al dataframe los datos de las zonas de subida y bajada. Para ello se va a realizar un join,
# De forma que vaya dentro del dataframe y sea accesible también desde un rdd

df_lookup = spark.read.csv('./taxi+_zone_lookup.csv', header=True, inferSchema=False)

datosLimpios = datosLimpios.withColumn(
    "LocationID", col("PULocationID")
).join(
    # Renombramos para poder luego incluir también las de bajada
    df_lookup.withColumnRenamed("Borough", "PUBorough"
            ).withColumnRenamed("Zone", "PUZone"
            ).withColumnRenamed("service_zone", "PUservice_zone"),
    on=['LocationID']
).withColumn(
    "LocationID", col("DOLocationID")
).join(
    df_lookup.withColumnRenamed("Borough", "DOBorough"
            ).withColumnRenamed("Zone", "DOZone"
            ).withColumnRenamed("service_zone", "DOservice_zone"),
    on=['LocationID']
)

datosLimpios.createOrReplaceTempView('datosCarrerasLimpios')

# datosLimpiosP = datosLimpios.toPandas()
# display(datosLimpiosP)
# display(datosLimpiosP.describe().T)
t_limpieza = time() - t0

### Velocidad media de los taxis

En este apartado se realizará un análisis de la velocidad media de los taxis, para ello se realizará una transformación de millas a metros sabiendo que 1 milla = 1609.344 metros luego dividiéndolo entre la diferencia de tiempo calculada previamente.

In [ ]:
if MODO_JAJAS:
    display(IFrame("https://www.youtube.com/embed/4vlVN5r7sKg", width="360", height="202"))

In [ ]:
t0velocidades = time()
dfMTS = datosLimpios.withColumn(
    "mean_speed", col("trip_distance")*1609.344/col("time_diff")
)
dfMTS.createOrReplaceTempView('datosCarrerasLimpiosConVelocidad')

In [ ]:
dfMTSP = dfMTS.toPandas()
t1velocidades = time()
display(dfMTSP.sort_values(by=["mean_speed"],ascending=False).head(50))
display(dfMTSP.describe().T)

En vista de que las velocidades medias estaban mal debido a que el time_diff es muy bajo, probablemente por un error de los tiempos almacenador por los taxistas, se volverá a realizar una consulta eliminando tiempos menores a 3 minutos y se comprobará las velocidades promedio otra vez.


In [ ]:
t2velocidades = time()
datosLimpiosSinVelocidades = spark.sql("SELECT * FROM datosCarrerasLimpios where time_diff >= 180")
dfMTS = datosLimpiosSinVelocidades.withColumn(
    "mean_speed", col("trip_distance")*1609.344/col("time_diff")
)

In [ ]:
dfMTSP = dfMTS.toPandas()
t3velocidades = time()
display(dfMTSP.sort_values(by=["mean_speed"],ascending=False).head(10))
display(dfMTSP.describe().T)



Como se puede observar, la mayoría de velocidades entre las 50 más rápidas superan el límite de velocidad nacional para zonas de carretera (24.72222 m/s) siendo que solo los 5 últimos lo cumplen, o en otras palabras que los 45 primeros infringen la ley.

Por otro lado se puede ver que los 8 primeros tienen velocidades mayores a 52 metros por segundo, lo que implica velocidades de 187.2 km/s esto puede ser debido a que haya algún tipo de fallo en el tiempo o que lleve velocidades demasiado altas.

Por último mencionar que los 6 primeros tienen velocidades mayores a 65 m/s, cosa que ya debe ser debido a un fallo, accidental o adrede por parte del conductor.


### Gráfica velocidad media por hora

In [ ]:
t4velocidades = time()
velocidades_por_hora = spark.sql("""
    SELECT mean_speed, HOUR(tpep_dropoff_datetime) as hour 
      FROM datosCarrerasLimpiosConVelocidad
      WHERE mean_speed <50
""")

velocidades_hora = velocidades_por_hora.groupBy("hour").mean("mean_speed").collect()

t5velocidades=time()

velocidades_x = [round(h,2) for h, cnt in velocidades_hora]
velocidades_y = [round(cnt*3.6,1) for h, cnt in velocidades_hora]

fig = plt.figure(figsize=(30, 10))

ax = fig.add_subplot(1, 1, 1)
rects1 = ax.bar(velocidades_x, velocidades_y)
ax.set_xlabel("Horas", fontsize=25)
ax.set_ylabel("Velocidades (kmh)", fontsize=25)
ax.set_title("Velocidades medias por hora", fontsize=25)
plt.xticks(rotation=90, fontsize=25)
plt.yticks(fontsize=25)
autolabel(rects1)

plt.show()

Tras realizar una grafica de velocidades medias por hora se puede observar que cuando las velocidades mas altas son a las horas nocturnas, probablemente porque hay menos trafico y menos vigilancia. 

In [ ]:
t_velocidad_media=t1velocidades-t0velocidades+t3velocidades-t2velocidades+t5velocidades-t4velocidades

### Zonas de poca cobertura

Observando la variable `store_and_fwd_flag`, creemos que es posible deducir qué zonas de la ciudad de Nueva York dan un mayor problema a la hora de estar conectados con el servidor de la compañía de taxis, es decir, tienen poca cobertura.

In [ ]:
if MODO_JAJAS:
    display(IFrame("https://giphy.com/embed/PmdOx0iRRtqkBFlEgI", width="240", height="240"))

In [ ]:
t0_cobertura = time()
sinCobertura_rdd = spark.sql("""
    SELECT DOBorough, DOZone
      FROM datosCarrerasLimpios
      WHERE store_and_fwd_flag == 'Y'
""").rdd

# sC_rdd.flatMap(lambda x: x['locationID']).map(lambda x: (x,1))
zone_tuples = sinCobertura_rdd.map(
    lambda x: (x['DOZone'],1)
).reduceByKey(
    lambda x,y: x+y
).sortBy(
    lambda x: x[1], False
)
borough_tuples = sinCobertura_rdd.map(
    lambda x: (x['DOBorough'],1)
).reduceByKey(
    lambda x,y: x+y
).sortBy(
    lambda x: x[1], False
)

In [ ]:
zonas = zone_tuples.take(10)
distritos = borough_tuples.take(10)

distritos_x = [k for k, v in distritos]
distritos_y = [v for k, v in distritos]
zonas_x = [k for k, v in zonas]
zonas_y = [v for k, v in zonas]

fig = plt.figure(figsize=(30, 10))

ax = fig.add_subplot(1, 2, 1)
rects1 = ax.bar(distritos_x, distritos_y)
ax.set_xlabel("Distrito", fontsize=25)
ax.set_ylabel("Registros guardados", fontsize=25)
ax.set_title("Registros guardados por distritos", fontsize=25)
plt.xticks(rotation=90, fontsize=25)
plt.yticks(fontsize=25)
autolabel(rects1)

ax = fig.add_subplot(1, 2, 2)
rects2 = ax.bar(zonas_x, zonas_y)
ax.set_xlabel("Zona", fontsize=25)
ax.set_ylabel("Registros guardados", fontsize=25)
ax.set_title("Registros guardados por zonas", fontsize=25)
plt.xticks(rotation=90, fontsize=25)
plt.yticks(fontsize=25)
autolabel(rects2)

plt.show()

In [ ]:
viajes_manhattan = spark.sql("""
    SELECT VendorID
      FROM datosCarrerasLimpios
      WHERE DOBorough == 'Manhattan'
""").count()
int(distritos_y[0])/int(viajes_manhattan)*100

Como se puede apreciar por el número de registros guardados en los taxis, Manhattan es el distrito que tiene más registros guardados con 2490, casi 5 veces más que su distrito posterior, Queens con 435. Por lo que se puede deducir que tiene zonas frecuentadas en las que no se tiene cobertura.

Sin embargo, es un porcentaje pequeño sobre el total, ya que representan menos del 0.3% de todos los viajes existentes que finalizan en Manhattan, por lo que muy probablemente se deba a taxis concretos que tienen problemas de conexión.

Observando que se trata de un porcentaje muy bajo, surge otra pregunta: ¿En qué momento se guardaron los registros? ¿Todos los registros guardados son en horas similares y por tanto puede deberse a una caída del servidor más que a un problema de cobertura?

Para ello se van a analizar los registros de Manhattan ya que son los más numerosos. Para ello nos quedaremos con el día en el que se han guardado el mayor número de mensajes y se analizará en qué horas se han guardado los mensajes, ya que de ser una caída del servidor, deberían agruparse alrededor de una hora concreta.

In [ ]:
stored_manhattan = spark.sql("""
    SELECT tpep_dropoff_datetime, DAY(tpep_dropoff_datetime) as day, HOUR(tpep_dropoff_datetime) as hour
      FROM datosCarrerasLimpios
      WHERE store_and_fwd_flag == 'Y' and DOBorough == 'Manhattan'
""")

# Nos vamos a quedar con el día con mayor número de registros guardados
day, count = stored_manhattan.groupBy("day").count().sort("count").tail(1)[0]

# Ahora que tenemos el día, podemos ver las horas en las que se agruparon los mensajes.
# Si todos (o casi todos) los mensajes se agruparon en la misma hora, eso significa 
# (muy probablemente) que hubo una caída del servidor
mensajes_hora = stored_manhattan.filter(col("day")==day).groupBy("hour").count().collect()
t_cobertura = time() - t0_cobertura

mensajes_x = [h for h, cnt in mensajes_hora]
mensajes_y = [cnt for h, cnt in mensajes_hora]

fig = plt.figure(figsize=(30, 10))

ax = fig.add_subplot(1, 1, 1)
rects1 = ax.bar(mensajes_x, mensajes_y)
ax.set_xlabel("Horas", fontsize=25)
ax.set_ylabel("Mensajes", fontsize=25)
ax.set_title("Mensajes guardados por hora", fontsize=25)
plt.xticks(rotation=90, fontsize=25)
plt.yticks(fontsize=25)
autolabel(rects1)

plt.show()

print(f"""
    Se ha tardado {round(t_cobertura, 2)}s en realizar este estudio sobre {datosLimpios_count},
    lo que da un procesamiento de {round(datosLimpios_count/t_cobertura, 2)} dato/seg
""")

Los mensajes guardados se agrupan entre las 10 AM y las 21 PM, pero dado que se corresponde también con el periodo de mayor actividad de los taxis, ya que es cuando la gran mayoría de la población se desplaza por la ciudad, no parece que se deba a una caída del servidor.

Por lo tanto nos reafirmamos en la primera suposición: **Muy probablemente se debe a taxis concretos que tienen problemas de conexión**, no a una caída generalizada. Además si hubiese sido ese el caso, el número de registros debería haber sido muchísimo mayor

### Viajes más comunes

En este apartado se realizará un estudio de los viajes más comunes a realizarse, para esto se tomarán del dataframe los datos referentes a las zonas de los viajes y con esto se hará un modelo sencillo de map-reduce para ver la cantidad de cada viaje.

In [ ]:
t0viajesComunes = time()
viajes_comunes = spark.sql("""
    SELECT PUZone, DOZone, PUBorough, DOBorough
      FROM datosCarrerasLimpios 
      WHERE PUBorough != 'Unknown' or DOBorough != 'Unknown'
""")
viajes_comun=viajes_comunes.rdd.map(
    lambda x: (f"De {x['PUZone']}({x['PUBorough']}) a {x['DOZone']}({x['DOBorough']})", 1)
).reduceByKey(
    lambda a,b: a + b
).sortBy(
    lambda x: x[1], False
)

Una aspecto a mencionar antes de continuar es que el viaje mas comun es el de Unknown a Unknown, cosa que no aporta demasiada informacion, por lo tanto estos viajes no se han tenido en cuenta de cara al analisis de los datos.

Tras realizar la organización de los datos en tablas y realizar la reducción, se tomarán los 10 viajes más comunes y se
graficarán como un gráfico de barras horizontal para poder visualizar las diferencias entre la cantidad de estos viajes.


In [ ]:
grafica = viajes_comun.take(10)
t1viajesComunes = time()    
x, y = [],[]
for i in grafica:    
    x.append(i[0]),y.append(i[1])
    
plt.figure()
plt.barh(x,y)
plt.gca().invert_yaxis()

In [ ]:
t_viajes_comunes=t1viajesComunes-t0viajesComunes

## Propinas

Ahora se va a realizar un estudio sobre las propinas que dejan los pasajeros.
Vamos a empezar contabilizando el numero de propinas que se dejan y clasificarlos por hora, para así poder ver en qué momento del día es más común que se dejen propinas.

In [ ]:
t0Propinas=time()

propinas_hora_query=spark.sql("""
  SELECT tip_amount, HOUR(tpep_dropoff_datetime) as hour 
      FROM datosCarrerasLimpios
""")

propinas_por_hora = propinas_hora_query.groupBy("hour").count().collect()

t1Propinas=time()

propinas_x = [h for h, cnt in propinas_por_hora]
propinas_y = [cnt for h, cnt in propinas_por_hora]

fig = plt.figure(figsize=(30, 10))

ax = fig.add_subplot(1, 1, 1)
rects1 = ax.bar(propinas_x, propinas_y)
ax.set_xlabel("Horas", fontsize=25)
ax.set_ylabel("Numero de propinas", fontsize=25)
ax.set_title("Numero de propinas por hora", fontsize=25)
plt.xticks(rotation=90, fontsize=25)
plt.yticks(fontsize=25)
autolabel(rects1)

plt.show()


Como podemos ver, las horas más comunes son las cercanas a las 18h, es decir, a media tarde coincidiendo con la hora de salir de trabajar.

Ahora se va a realizar un estudio de la cantidad de dinero total (en miles de dólares) dejado en cada periodo para ver su similitud con la cantidad de propinas dejadas por hora.

In [ ]:
t2Propinas=time()
propinas_totales_por_hora = propinas_hora_query.groupBy("hour").sum('tip_amount').collect()

t3Propinas=time()

propinas_x = [h for h, cnt in propinas_totales_por_hora]
propinas_y = [round(cnt/1000) for h, cnt in propinas_totales_por_hora]

fig = plt.figure(figsize=(30, 10))

ax = fig.add_subplot(1, 1, 1)
rects1 = ax.bar(propinas_x, propinas_y)
ax.set_xlabel("Horas", fontsize=25)
ax.set_ylabel("Propinas totales(k$)", fontsize=25)
ax.set_title("Propinas totales(k$) por hora", fontsize=25)
plt.xticks(rotation=90, fontsize=25)
plt.yticks(fontsize=25)
autolabel(rects1)

plt.show()

Como podemos ver, la gráfica resultante es similar a la anterior, por lo que vemos que la cantidad de propinas dejadas tiene relación con la aportación monetaria que producen.

Sin embargo, ahora vamos a hacer un estudio para ver en qué momento del día se dejan las propinas más cuantiosas.

In [ ]:
t4Propinas=time()
propinas_maximas_por_hora = propinas_hora_query.groupBy("hour").max('tip_amount').collect()

t5Propinas=time()

propinas_x = [h for h, cnt in propinas_maximas_por_hora]
propinas_y = [cnt for h, cnt in propinas_maximas_por_hora]

fig = plt.figure(figsize=(30, 10))

ax = fig.add_subplot(1, 1, 1)
rects1 = ax.bar(propinas_x, propinas_y)
ax.set_xlabel("Horas", fontsize=25)
ax.set_ylabel("Propinas maximas($)", fontsize=25)
ax.set_title("Propinas maximas($) por hora", fontsize=25)
plt.xticks(rotation=90, fontsize=25)
plt.yticks(fontsize=25)
autolabel(rects1)

plt.show()

Como podemos ver, hay propinas muy altas a las horas más comunes, pero otro conjunto de picos en la gráfica, nos indica que de madrugada se dejan también propinas muy cuantiosas, aunque no sean tan numerosas como a media tarde.

In [ ]:
tiempoPropinasTotal=(t5Propinas-t4Propinas)+(t3Propinas-t2Propinas)+(t1Propinas-t0Propinas)
print(tiempoPropinasTotal,"Segundos en realizarse el estudio de propinas.")

# Resumen de tiempos, explicación y conclusiones

En esta tabla se recogen todos los tiempos calculados previamente en los estudios realizados.

In [ ]:
print(f"""
{"-"*150}
\t\tEstudio\t\t|\tTiempo\t\t|\tVelocidad procesamiento\t\t| Nº Transformaciones-Acciones\t|\tRDD/SQL 
{"-"*150}
{"-"*150}
\tLimpieza\t\t|\t{round(t_limpieza, 2)} s\t\t|\t\t{round(initial_data_count/t_limpieza, 2)}\t\t|\t\t3-4\t\t|\tSQL
{"-"*150}
\tVelocidad media taxis\t|\t{round(t_velocidad_media, 2)} s\t\t|\t\t{round(cantidad_velocidad_media/t_velocidad_media, 2)}\t\t|\t\t4-2\t\t|\tSQL
{"~"*150}
\tZonas sin cobertura\t|\t{round(t_cobertura, 2)} s\t\t|\t\t{round(datosLimpios_count/t_cobertura, 2)}\t\t|\t\t10-8\t\t|\tHibrido
{"~"*150}
\tPropinas\t\t|\t{round(tiempoPropinasTotal, 2)} s\t\t|\t\t{round(datosLimpios_count/tiempoPropinasTotal, 2)}\t\t|\t\t5-4\t\t|\tSQL
{"~"*150}
\tViajes más comunes\t|\t{round(t_viajes_comunes, 2)} s\t\t|\t\t{round(c_viajes_comunes/t_viajes_comunes, 2)}\t\t|\t\t3-2\t\t|\tHibrido
{"~"*150}
""")

Como podemos ver, tenemos tiempos bastante dispares entre los distintos estudios. El más lento es el de Zonas sin cobertura y el más rápido el de Propinas. La diferencia de tiempos se debe principalmente a la complejidad tanto de los propios estudios como de las querys que en ellos se realizan. En este caso, en la más costosa se realizan varias acciones y transformaciones para sacar los registros guardados por distryto y por zona, después se busca cual es el día con más registros en el distrito con más registros para obtener la cantidad de mensajes guardados por hora. Ese estudio es de mayor complejidad que obtener las cantidades de proinas por hora, cantidad de éstas y el máximo en cada franja horaria.

# Fin

In [ ]:
#spark.stop()